## ChromaDB を使った RAG のデモ

### OpenAI API Key とLLM、埋め込みモデルの設定

In [1]:
from dotenv import load_dotenv, find_dotenv
# 環境変数 OPENAI_API_KEY に OpenAI API Key を設定（.env ファイルの OPENAI_API_KEY からロード）
_ = load_dotenv(find_dotenv())
embedding_model = "text-embedding-3-large" # 埋め込みモデル
model = "gpt-4o" # LLM
temperature = 0 # LLMの生成のランダムさ

### LLMと埋め込みモデルの準備

In [2]:
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings

# LLMの準備
llm = ChatOpenAI(
    model=model, # モデル
    temperature=temperature, # ランダムさ
)

# 埋め込みモデルの準備
embeddings = OpenAIEmbeddings(model=embedding_model)

### 検索対象の ChromaDB コレクション名の設定
コレクションとは、ChromaDB におけるベクトルデータの保存単位で、RDB のテーブルのようなもの

In [3]:
#collection_name = "mycollection_100" # 検索対象のChromadb コレクション名（チャンクサイズ100文字で事前に作成済み）
collection_name = "mycollection_200" # 検索対象のChromadb コレクション名（チャンクサイズ200文字で事前に作成済み）
#collection_name = "mycollection_500" # 検索対象のChromadb コレクション名（チャンクサイズ500文字で事前に作成済み）
#collection_name = "mycollection_1000" # 検索対象のChromadb コレクション名（チャンクサイズ1000文字で事前に作成済み）

### VectorStoreの準備

In [4]:
import chromadb
from langchain_chroma import Chroma

# LangChain の Chromadb VectorStore のインスタンスを作成
vector_store = Chroma(
    client=chromadb.PersistentClient(path="./chroma_db"),
    collection_name=collection_name,
    embedding_function=embeddings,
    collection_metadata={"hnsw:space": "ip"} 
)

### PromptTemplateの準備

In [5]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "あなたの仕事は、human の question に答えることです。以下の手順で回答してください。前置きや思考経過は出力しないでください。context の中の source のファイル名には、content の主題が含まれています。最初に、 context に書かれている情報だけを使用して、question の質問に答えることができるかどうか判断してください、判断結果は出力しません。次に、context にある情報だけで回答が可能と判断した場合は、context に書かれている情報だけを使用して、question の質問に答えてください。context にある情報だけでは回答できないと判断した場合は、情報がないので回答できないと答えてください。\n\ncontext: {context}"),
        ("human", "question: {input}"),
    ]
)

### 質問の定義

In [15]:
query="フェイトが留学生として通った小学校は？"

### Retrieverの準備（フィルターなし）

In [16]:
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 20}
)

In [17]:
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    {"context": retriever, "input": RunnablePassthrough()}
    | prompt_template
    | (lambda x: print(f"\n=== Generated Prompt ===\n{x}\n========================\n\n") or x)  # プロンプトを表示
    | llm
)

### RAGチェーンの準備

In [18]:
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    {"context": retriever, "input": RunnablePassthrough()}
    | prompt_template
    | (lambda x: print(f"\n=== Generated Prompt ===\n{x}\n========================\n\n") or x)  # プロンプトを表示
    | llm
)

### RAGの実行（フィルターなし）

In [19]:
response = rag_chain.invoke(query)
print(f"=== LLM response ======\n{response}\n========================\n")
print(f"=== 回答 ===============\n{response.content}\n========================\n")


=== Generated Prompt ===
messages=[SystemMessage(content='あなたの仕事は、human の question に答えることです。以下の手順で回答してください。前置きや思考経過は出力しないでください。context の中の source のファイル名には、content の主題が含まれています。最初に、 context に書かれている情報だけを使用して、question の質問に答えることができるかどうか判断してください、判断結果は出力しません。次に、context にある情報だけで回答が可能と判断した場合は、context に書かれている情報だけを使用して、question の質問に答えてください。context にある情報だけでは回答できないと判断した場合は、情報がないので回答できないと答えてください。\n\ncontext: [Document(id=\'279f9053-60af-40f8-8585-3ffd978d5351\', metadata={\'source\': \'data\\\\Wikipedia-高町なのは.pdf\'}, page_content=\'魔法を知ってからはさらに良くなっている)ただし文系(アリサ曰く中の下)と体育が苦手と本人はいっているがどれほどなのかははっきりしない。なのはの住む世界では非常に珍しく、魔導師として「天才」と呼べる素質があり、更に希少なレアスキルである『魔力収束』を持ち、ユーノを師として実戦を繰り返す中で急速に才能を開花させてゆく。魔力の放出‧集束と制御を得意とし、圧縮‧縮小は苦手。\'), Document(id=\'7cf792fd-e8b9-4feb-a948-3277ae28c83f\', metadata={\'source\': \'data\\\\Wikipedia-魔法科高校の劣等生.pdf\'}, page_content=\'九校戦の新入生女子代表メンバー。スピードシューティングでは雫、英美に次ぐ三位の成績を収めている。『魔法科高校の優等生』では達也の不敗記録が最も破られかねない試合で、実力上位である第三高校の十七夜 栞に三位決定戦で勝っている。春日 菜々美(かすが ななみ)声 - 久野美咲2095年の九校戦の選手に選ばれた、一科生の

### Retrieverの準備（フィルターあり）

In [11]:
source_list=["data\\Wikipedia-フェイト・テスタロッサ.pdf"]

In [12]:
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 20, "filter": {"source": {"$in": source_list}}}
)

### RAGチェーンの準備

In [13]:
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    {"context": retriever, "input": RunnablePassthrough()}
    | prompt_template
    | (lambda x: print(f"\n=== Generated Prompt ===\n{x}\n========================\n\n") or x)  # プロンプトを表示
    | llm
)

### RAGの実行（フィルターあり）

In [14]:
response = rag_chain.invoke(query)
print(f"=== LLM response ======\n{response}\n========================\n")
print(f"=== 回答 ===============\n{response.content}\n========================\n")



=== Generated Prompt ===
messages=[SystemMessage(content='あなたの仕事は、human の question に答えることです。以下の手順で回答してください。前置きや思考経過は出力しないでください。context の中の source のファイル名には、content の主題が含まれています。最初に、 context に書かれている情報だけを使用して、question の質問に答えることができるかどうか判断してください、判断結果は出力しません。次に、context にある情報だけで回答が可能と判断した場合は、context に書かれている情報だけを使用して、question の質問に答えてください。context にある情報だけでは回答できないと判断した場合は、情報がないので回答できないと答えてください。\n\ncontext: [Document(id=\'dc122ab2-0305-4855-91eb-e11d08dac400\', metadata={\'source\': \'data\\\\Wikipedia-フェイト・テスタロッサ.pdf\'}, page_content=\'さらに、手足にバルディッシュのフィンブレードやなのはのフライアーフィンのような光の羽「ソニックセイル」を生やしている。また、右手にも装甲が追加されている。圧倒的な運動性‧機動性‧攻撃速度を手に入れた分、「受け」に使用する手足以外は防御力は無いに等しく、加速と攻撃の反動に耐える以外の目的は無い。攻撃に当たれば致命傷になりかねないのでまさに諸刃の剣といってよい。\'), Document(id=\'cdaca5ab-e317-4418-ba64-3f5b839fc996\', metadata={\'source\': \'data\\\\Wikipedia-フェイト・テスタロッサ.pdf\'}, page_content=\'なのは同様魔導師としての才能は非凡なものがある上に、幼い頃から母の使い魔であったリニスから戦闘訓練を受けて驚異的な速度でその教えを習得。出会った当初、なのはは手も足も出なかった。高速移動からの斬撃による一撃離脱を得意とし、射撃‧広範囲魔法も優れた前衛戦闘型の魔導師。一方でバリア出力の低さなど防御面に難があ